<div style="border:solid green 4px; padding: 20px">Hello! My name is Zaur. My critical comments are highlighted with <span style='color: red;'>red</span>,  less urgent remarks are in <span style='color: #ebd731;'>yellow</span>, recommendations and extra information - in <span style='color: green;'>green</span>.</div>

## Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

### Step 1. Open the data file and have a look at the general information. 

In [1]:
import pandas as pd
#importing the dataset
x = pd.read_csv('/datasets/credit_scoring_eng.csv')
#cursory glances at what our data looks like
x.head()
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Conclusion

There are inconsistencies in the type of quantatative variables; some are int64 and some are float64. We might want to convert them all to the same type. Some string types, like gender could be coded as a integer too, since all the gender is either male or female. There are also some glaring missing values.

### Step 2. Data preprocessing

### Processing missing values

In [2]:
col = x.columns
#print(col)

#for i in col:
    #print(x[i].isna().count())
    
#making a vector of the names of the columns that are of string type   
str_col = ['education', 'family_status','gender', 'income_type','purpose']

#Filling empty cells with a blank string
for i in str_col:
    x[i] = x[i].fillna(value='')
    #print(x[i].value_counts())
    
#making a vector of the names of the columns that are of numeric type     
num_avg_col = ['children','days_employed','dob_years','total_income']
#Filling empty cells with a the average of that column
for i in num_avg_col:
    avg = x[i].mean()
    x[i] = x[i].fillna(value=avg)
    
print(x['children'].value_counts())

#for i in x['children']:
 #   x.loc[(x['children'] < 0), 'children']= None
  #      
#print(x['children'].value_counts())

#Deleting the rows where children are -1 years old
childindex = x[x['children'] < 0].index
x.drop(childindex,inplace = True)
print(x['children'].value_counts())


 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
0     14149
1      4818
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64


<div style="border:solid green 4px; padding: 20px">Is there any difference for us if a string value is null or ""?</div>

### Conclusion

For columns that contained strings, empty cells were replaced with an empty string. For columns that were numbers that made sense to be represented as averages in the event of no data, that's what I did. The children column has 47 people with -1 children, so I made those values 'null'. But later in the code when I tried converting it to integer, I encountered a bug. Since these are only 47 rows, I decided to delete these rows. As to why we have this data, I am not sure since it doesn't seem like a human data entry error, so in some step along the way this data might have got corrupted.

### Data type replacement

In [3]:
#Making a vector of all the columns that are of type float
float_col = ['children','days_employed', 'total_income']
#Converting all float columns to int
for i in float_col:
    x[i] = x[i].astype(int)
    
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21478 entries, 0 to 21524
Data columns (total 12 columns):
children            21478 non-null int64
days_employed       21478 non-null int64
dob_years           21478 non-null int64
education           21478 non-null object
education_id        21478 non-null int64
family_status       21478 non-null object
family_status_id    21478 non-null int64
gender              21478 non-null object
income_type         21478 non-null object
debt                21478 non-null int64
total_income        21478 non-null int64
purpose             21478 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


<div style="border:solid #ebd731; 4px; padding: 20px">You can just apply .astype() to any number of columns with a single slice:<br>
    x[float_col] = x[float_col].astype(int)</div>

### Conclusion

We successfully converted all the float columns to integer columns. We also see in the info output that there are no non-null objects.

### Processing duplicates

In [10]:
#First, we convert all the strings to lowercase
for i in str_col:
    x[i] = x[i].str.lower()
    #print(x[i].value_counts())
#Dropping rows that have the same value for the pararmeters we specified
x.drop_duplicates(['children','days_employed','dob_years','total_income'],keep= 'last')
print(x.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21478 entries, 0 to 21524
Data columns (total 15 columns):
children            21478 non-null int64
days_employed       21478 non-null int64
dob_years           21478 non-null int64
education           21478 non-null object
education_id        21478 non-null int64
family_status       21478 non-null object
family_status_id    21478 non-null int64
gender              21478 non-null object
income_type         21478 non-null object
debt                21478 non-null int64
total_income        21478 non-null int64
purpose             21478 non-null object
kids                21478 non-null bool
income_group        21478 non-null object
purpose_type        20982 non-null object
dtypes: bool(1), int64(7), object(7)
memory usage: 2.5+ MB
None


### Conclusion

We converted all strings to lowercase since there were strings that were the exact same but just written with different cases for certain letters. We do not have a uniquie identifier for each person, so we will have to assume that the probability of two people have the same children,days employed, dob, and total income is negligible and consider them duplicates. Using this criterion, it seems like there are no duplicates since the drop duplicates method has not reduced the number of rows.

### Categorizing Data

In [12]:
print(x['family_status'].value_counts())
print(x['children'].value_counts())

#creating a boolean vector where true means you have kids and false means you do not
x['kids'] = x['children']>0

#Finding the 1st and 3rd quartiles of the total income to find a suitable cut off for the income brackets
q1 = x['total_income'].quantile(0.25)
q3 = x['total_income'].quantile(0.75)

#categorizing the income groups
def income_group(income):
    
    if income <= q1:
        return 'low income'
    if income <= q3:
        return 'mid income'
    return 'high income'


x['income_group'] = x['total_income'].apply(income_group)
print(x['income_group'].value_counts())

#categorizing the purposes based on what words they contain.
#Took help from Lindsey Naylor and Aleksandr Kravtsov on the Slack channel to write this part
def group_purpose(purp):
    wedding_words = ['wedding']
    real_estate_words = ['real estate', 'transaction', 'transactions', 'property']
    home_words = ['house', 'housing']
    car_words = ['car', 'cars']
    school_words = ['education', 'educated']
    
    for wedding_word in wedding_words:
        if wedding_word in purp:
            return 'wedding'
    for real_estate_word in real_estate_words:
        if real_estate_word in purp:
            return 'real estate'
    for home_word in home_words:
        if home_word in purp:
            return 'house'
    for car_word in car_words:
        if car_word in purp:
            return 'car'
    for school_word in school_words:
        if school_word in purp:
            return 'education'
x['purpose_type'] = x['purpose'].apply(group_purpose)

print(x['purpose_type'].value_counts())

married              12351
civil partnership     4172
unmarried             2808
divorced              1191
widow / widower        956
Name: family_status, dtype: int64
0     14149
1      4818
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64
mid income     10737
low income      5371
high income     5370
Name: income_group, dtype: int64
real estate    7655
car            4304
education      3516
house          3161
wedding        2346
Name: purpose_type, dtype: int64


<div style="border:solid green 4px; padding: 20px">I really appreciate you mentioning your helpers 😃</div>

<div style="border:solid #ebd731; 4px; padding: 20px">Look <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html">this</a>.</div>

### Conclusion

Marital status is already categorized, having children or not has a binary column and 'purpose' has been grouped into categories based on how similar they were. I used the 1st and 3rd quartiles to create 3 bins for income levels.

### Step 3. Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [14]:
#Creating a pivot table and a column that is the ratio of the debt to no-debt column
piv = x.pivot_table(index=['kids'],columns = 'debt', values = 'children', aggfunc = 'count')
total = piv.sum().sum()
piv['debt - no debt ratio'] = piv[1]/piv[0]*100
piv['debt ratio'] = piv[1]/total*100
print(piv)


debt       0     1  debt - no debt ratio  debt ratio
kids                                                
False  13086  1063              8.123185    4.949250
True    6652   677             10.177390    3.152063


<div style="border:solid green 4px; padding: 20px">Good. You know, by the nature of "debt" being binary - mean of that column gives the same needed result (shares).</div>

### Conclusion

The value of 8.12 as compared to 10.18 suggests that people with kids are more likely to be in debt that people without kids.

- Is there a relation between marital status and repaying a loan on time?

In [7]:
#Creating a pivot table and a column that is the ratio of the debt to no-debt column
piv = x.pivot_table(index=['family_status_id'],columns = 'debt', values = 'family_status', aggfunc = 'count')
total = piv.sum().sum()
piv['debt - no debt ratio'] = piv[1]/piv[0]*100
#piv['debt ratio'] = piv[1]/total*100
#piv_new = piv.rename(columns = {'0': 'No debt', '1':'Debt'}, index={'False': 'No Kids','True':'Have kids'})
print(piv)


debt                  0    1  debt - no debt ratio
family_status_id                                  
0                 11421  930              8.142895
1                  3784  388             10.253700
2                   893   63              7.054871
3                  1106   85              7.685353
4                  2534  274             10.812944


### Conclusion

Unmarried and civil partnership are more likely to be in debt as compared to the other marital statuses.

- Is there a relation between income level and repaying a loan on time?

In [8]:
#Creating a pivot table and a column that is the ratio of the debt to no-debt column
piv = x.pivot_table(index=['income_group'],columns = 'debt', values = 'total_income', aggfunc = 'count')
total = piv.sum().sum()
piv['debt - no debt ratio'] = piv[1]/piv[0]*100
#piv['debt ratio'] = piv[1]/total*100
#piv_new = piv.rename(columns = {'0': 'No debt', '1':'Debt'}, index={'False': 'No Kids','True':'Have kids'})
print(piv)

debt             0    1  debt - no debt ratio
income_group                                 
high income   4985  385              7.723170
low income    4944  427              8.636731
mid income    9809  928              9.460699


### Conclusion

It seems like mid income group is most susceptible to being in debt. Low income is slightly less and high income group is least likely to be in debt.

- How do different loan purposes affect on-time repayment of the loan?

In [9]:
#Creating a pivot table and a column that is the ratio of the debt to no-debt column
piv = x.pivot_table(index=['purpose_type'],columns = 'debt',values = 'purpose', aggfunc = 'count')
total = piv.sum().sum()
piv['debt - no debt ratio'] = piv[1]/piv[0]*100
#piv['debt ratio'] = piv[1]/total*100
#piv_new = piv.rename(columns = {'0': 'No debt', '1':'Debt'}, index={'False': 'No Kids','True':'Have kids'})
print(piv)

debt             0    1  debt - no debt ratio
purpose_type                                 
car           3902  402             10.302409
education     3189  327             10.253998
house         2953  208              7.043684
real estate   7081  574              8.106200
wedding       2160  186              8.611111


### Conclusion

People with purposes related house are least likely to be in debt.

### Step 4. General conclusion

Overall, it seems like loan purpose, marital status, income level and whether you have kids or not are good indicators of how likely it is that you are in debt. We can similarly look at the other attributes like gender and see if there is conclusive evidence that suggests how likely you are to go into debt (of course, the ethics of making decisions based on something like gender is something that should be considered too). Further directions could include having a way to incorporate all these factors by giving each parameter a weight and calculating an overall likelihood of being in debt based on that. 

### Project Readiness Checklist

Put 'x' in the completed points. Then press Shift + Enter.

<div style="border:solid green 4px; padding: 20px">Okay, well done. I like that your conclusions are neat and succinct.<br>
Although I had some recommendations, as you see, I appreciate your time and effort and look forward to see you going into further projects. Revise my commentary to use it next time. See you!</div>